In [1]:
import glob
import pandas as pd
import numpy as np
import re
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderUnavailable
import time
import math
from progressbar import progressbar


# Data import and primary data curation

**Author:** Tobias Schulze
**Last update:** 16 July 2023

All imported files were preprocessed to standardize file name schemes and separators.

In general, issues occurring during import and data type conversion was fixed on the fly.

Further curation will be performed after in deep evaluation of the data.


## Purpose of this notebook

- Import the distributed data files from `../mar23cds_accidents_data_import` folder (only local to save space)

- Merge the data and save the merged tables per source in `./data`

- Fix general data issues:

    - Misspelled geolocation data
 
        - Replace the weird [0, 40] geolocation by NA
     
        - Replace all misspelled locations (e.g. [0, 0] which is not in France) by NA
     
        - Trial to impute missing georeferences on address data
    
    - Fix issues in `dep` variable
 
    - Fix other encoding issues

- Add a `metropolitan` variable with values `[0,1]` encoding metropolitan (continental France + Corse) and overseas departments (based on `dep`)

- Merge the Corse departments 2a and 2b to one department 2 to get an integer type `dep` variable

- Merge all data to a large dataframe

- Export dataframe for further processing

## `usagers` files
- import the files
- fix `dtype` issues on the fly
- fix an issue with the `id_vehicule` variable
- export the file to `users.csv` in `data`

### Data import and dtype issues fixing

In [2]:
# read file names
files = []
for file in glob.glob("../mar23cds_accidents_data_import/usagers*.csv"):
    files.append(file)

# import user files
users = pd.DataFrame()

for i in range(0, len(files)):
    file = pd.read_csv(files[i], sep = ',', index_col=0)
    if i == 0:
        users = file
    else:
        users = pd.concat([users, file], axis = 0)

# show info
users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2509620 entries, 201900000001 to 201800057783
Data columns (total 15 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id_vehicule  object 
 1   num_veh      object 
 2   place        float64
 3   catu         int64  
 4   grav         int64  
 5   sexe         int64  
 6   an_nais      float64
 7   trajet       float64
 8   secu1        float64
 9   secu2        float64
 10  secu3        float64
 11  locp         float64
 12  actp         object 
 13  etatp        float64
 14  secu         float64
dtypes: float64(9), int64(3), object(3)
memory usage: 306.4+ MB


### Fixing the issue in `id_vehicule`
We have a curious encoding in `id_vehicule`.

The value `\xa0` encodes a protected blank, which cannot be removed by integrated stripping functions.
Hence, we need a lambda function to:

1. remove all non-numeric characters
2. replace empty vales by nan
3. finally, convert the variable to integer

In [3]:
# show example
users.iloc[0].id_vehicule

'138\xa0306\xa0524'

In [4]:
# remove all non-numeric characters by a regex
users['id_vehicule'] = users['id_vehicule'].apply(lambda x: re.sub('[^0-9]', '', str(x)))

# replace empty values by nan
users['id_vehicule'] = users['id_vehicule'].replace('', np.nan).astype(float)


In [5]:
# check the result
users.iloc[0].id_vehicule

138306524.0

In [6]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2509620 entries, 201900000001 to 201800057783
Data columns (total 15 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id_vehicule  float64
 1   num_veh      object 
 2   place        float64
 3   catu         int64  
 4   grav         int64  
 5   sexe         int64  
 6   an_nais      float64
 7   trajet       float64
 8   secu1        float64
 9   secu2        float64
 10  secu3        float64
 11  locp         float64
 12  actp         object 
 13  etatp        float64
 14  secu         float64
dtypes: float64(10), int64(3), object(2)
memory usage: 306.4+ MB


### Export the data

In [7]:
users.to_csv("./data/users.csv", sep = ',', header = True, na_rep = 'n/a', index=True)

## `caracteristique` files
- import the files
- fix `dtype` issues on the fly
- fix issues with the variables `lat`, `long`, and `dep`
- export the file to `features.csv` in `data`

### Data import and dtype issues fixing

- encoding is `latin-1` to map French characters


In [8]:
#### read characteristics (=features) files in data directory
files = []
for file in glob.glob("../mar23cds_accidents_data_import/caracter*.csv"):
    files.append(file)

# import user files
features = pd.DataFrame()

for i in range(0, len(files)):
    file = pd.read_csv(files[i], sep = ',', encoding='iso-8859-1', na_values=['N/A', 'n/a',''], index_col=0)
    if i == 0:
        features = file
    else:
        features = pd.concat([features, file], axis = 0)


In [9]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1121571 entries, 200900000001 to 200500087954
Data columns (total 15 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   an      1121571 non-null  int64  
 1   mois    1121571 non-null  int64  
 2   jour    1121571 non-null  int64  
 3   hrmn    1121571 non-null  object 
 4   lum     1121571 non-null  int64  
 5   agg     1121571 non-null  int64  
 6   int     1121571 non-null  int64  
 7   atm     1121498 non-null  float64
 8   col     1121552 non-null  float64
 9   com     1121569 non-null  object 
 10  adr     978295 non-null   object 
 11  gps     480052 non-null   object 
 12  lat     634503 non-null   object 
 13  long    634499 non-null   object 
 14  dep     1121571 non-null  object 
dtypes: float64(2), int64(6), object(7)
memory usage: 136.9+ MB


In [10]:
features.head()

,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep
Num_Acc,,,,,,,,,,,,,,,
200900000001,9,1,30,2030,3,1,1,1.0,6.0,67,RTE DE GUï¿½MENï¿½,M,0.0,0,440
200900000002,9,1,17,300,3,1,1,1.0,3.0,17,LE BOIS JOLI,M,0.0,0,440
200900000003,9,1,29,645,3,1,1,1.0,3.0,52,NaN,M,0.0,0,440
200900000004,9,1,4,615,3,1,1,9.0,6.0,44,LE BECO,M,0.0,0,440
200900000005,9,1,16,1500,1,1,1,1.0,7.0,36,MOULIN DE LA GARENNE,M,0.0,0,440


In [11]:
# Pad the year to four numbers
features['an'] = [f'2{str(num).zfill(3)[-3:]}' if num < 10000 else str(num)[-4:] for num in features['an']]
features['an'] = pd.Series(features['an'], dtype='int32')

In [12]:
features.head()


,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep
Num_Acc,,,,,,,,,,,,,,,
200900000001,2009,1,30,2030,3,1,1,1.0,6.0,67,RTE DE GUï¿½MENï¿½,M,0.0,0,440
200900000002,2009,1,17,300,3,1,1,1.0,3.0,17,LE BOIS JOLI,M,0.0,0,440
200900000003,2009,1,29,645,3,1,1,1.0,3.0,52,NaN,M,0.0,0,440
200900000004,2009,1,4,615,3,1,1,9.0,6.0,44,LE BECO,M,0.0,0,440
200900000005,2009,1,16,1500,1,1,1,1.0,7.0,36,MOULIN DE LA GARENNE,M,0.0,0,440


In [13]:
features['an'].unique()

array([2009, 2013, 2007, 2011, 2018, 2012, 2010, 2014, 2008, 2016, 2019,
       2015, 2020, 2017, 2021, 2006, 2005], dtype=int32)

### The latitude and longitude data is formatted in two schemes:

1. decimal format with decimal separator
2. string format without decimal separator
3. NAs

The string format encodes the decimal format without decimals.
Hence, we have 3 data formats:

1. `float64`
2. `int64`
3. `NA`


In [14]:
# This part was optimised with help of ChatGPT 3.0
# remove all non-numeric characters by a regex
features['lat'] = features['lat'].apply(lambda x: re.sub('[^0-9]', '', str(x)))
features['long'] = features['long'].apply(lambda x: re.sub('[^0-9]', '', str(x)))

# split the string in two decimals before decimal separator for latitide and one in longitude
features['lat'] = features['lat'].apply(lambda x: f"{x[:2]}.{x[2:]}" if x and isinstance(x, str) else x)
features['long'] = features['long'].apply(lambda x: f"{x[:1]}.{x[1:]}" if x and isinstance(x, str) else x)

# add nan to all empty values
features['lat'] = features['lat'].replace('', np.nan).astype(float)
features['long'] = features['long'].replace('', np.nan).astype(float)

In [15]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1121571 entries, 200900000001 to 200500087954
Data columns (total 15 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   an      1121571 non-null  int32  
 1   mois    1121571 non-null  int64  
 2   jour    1121571 non-null  int64  
 3   hrmn    1121571 non-null  object 
 4   lum     1121571 non-null  int64  
 5   agg     1121571 non-null  int64  
 6   int     1121571 non-null  int64  
 7   atm     1121498 non-null  float64
 8   col     1121552 non-null  float64
 9   com     1121569 non-null  object 
 10  adr     978295 non-null   object 
 11  gps     480052 non-null   object 
 12  lat     634503 non-null   float64
 13  long    631290 non-null   float64
 14  dep     1121571 non-null  object 
dtypes: float64(4), int32(1), int64(5), object(5)
memory usage: 132.6+ MB


In [16]:
features.head()

,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep
Num_Acc,,,,,,,,,,,,,,,
200900000001,2009,1,30,2030,3,1,1,1.0,6.0,67,RTE DE GUï¿½MENï¿½,M,0.0,0.0,440
200900000002,2009,1,17,300,3,1,1,1.0,3.0,17,LE BOIS JOLI,M,0.0,0.0,440
200900000003,2009,1,29,645,3,1,1,1.0,3.0,52,NaN,M,0.0,0.0,440
200900000004,2009,1,4,615,3,1,1,9.0,6.0,44,LE BECO,M,0.0,0.0,440
200900000005,2009,1,16,1500,1,1,1,1.0,7.0,36,MOULIN DE LA GARENNE,M,0.0,0.0,440


## Fix department number issues

In [17]:
features['dep'].unique()

array([440, 750, 730, 160, 660, 220, 290, 850, 40, 270, 120, 70, 450, 80,
       480, 360, 600, 410, 280, 30, 430, 790, 680, 100, 140, 810, 50, 350,
       490, 870, 300, 830, 170, 860, 240, 540, 700, 510, 740, 460, 900,
       610, 530, 520, 210, 950, 340, 310, 560, 190, 10, 230, 840, 670,
       470, 370, 390, 640, 890, 590, 650, 380, 320, 400, 90, 150, 550,
       180, 880, 250, 570, 20, 201, 710, 330, 500, 780, 202, 770, 720,
       110, 60, 820, 800, 130, 940, 690, 260, 580, 910, 420, 920, 620,
       630, 930, 760, 971, 974, 972, 973, 976, '93', '92', '94', '87',
       '69', '38', '34', '13', '988', '976', '974', '972', '2B', '91',
       '86', '83', '80', '78', '77', '76', '72', '71', '67', '66', '64',
       '60', '51', '50', '49', '45', '37', '35', '33', '31', '30', '29',
       '22', '19', '18', '17', '74', '81', '2', '59', '95', '63', '62',
       '973', '2A', '84', '9', '73', '43', '10', '36', '16', '7', '21',
       '40', '24', '4', '85', '27', '28', '52', '68', '42', '82

Corse is separated in department 2a and 2b. We simplify it to 2. In addition, there are aliases 201 and 202.

In [18]:
# It contains 2A and 2B for Corse, hence, reduce Corse to department 2 in addition there is also 201 and 202 related to Corse
features.loc[features['dep'] == '2A', 'dep'] = 2
features.loc[features['dep'] == '2B', 'dep'] = 2
features.loc[features['dep'] == 201, 'dep'] = 2
features.loc[features['dep'] == 202, 'dep'] = 2
features['dep'] = features['dep'].astype(int)

The departments of France are numbered in the range 01 to 95 for metropolitan France and 971 to 976 for the abroad departments like Reunion.

However, departments above 95 and below 971 exist. We guess, that these are broken department numbers as many are marked by M or the location is in metropolitina France

In [19]:
features['dep'].sort_values().unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95, 100, 110, 120, 130, 140, 150, 160, 170, 180,
       190, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320,
       330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450,
       460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580,
       590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710,
       720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840,
       850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 97

In [20]:
# Strip the trailing zeros in the department 95 < x < 971
features['dep'] = features['dep'].apply(lambda x: str(x).rstrip('0') if 95 < x < 971 else x)

In [21]:
features['dep'] = features['dep'].astype(int)
features['dep'].sort_values().unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95, 971, 972, 973, 974, 975, 976, 977, 978, 986,
       987, 988])

### Mark abroad departments
[In general](https://www.map-france.com/departments), the numbers of continental departments are up to 95, those of overseas are in the range 971-976.

The larger departments, e.g. 978 are obviously locations in the French part of Canada.

Thus, let's remove datasets equal or greater as 95 to focus on metropolitan France.

In [22]:
# Mark the abroad and metropolitan locations
features.loc[features['dep'] <= 95, 'metropolitan'] = 1
features.loc[features['dep'] > 95, 'metropolitan'] = 0


In [23]:
features[features['dep'] > 95]

,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep,metropolitan
Num_Acc,,,,,,,,,,,,,,,,
200900072469,2009,1,3,1955,5,2,1,1.0,6.0,20,LAURICISQUE,NaN,NaN,NaN,971,0.0
200900072470,2009,1,29,1345,1,2,1,1.0,6.0,3,route nationale 2002,A,0.0,0.0,971,0.0
200900072471,2009,1,25,2015,1,2,1,1.0,3.0,27,rue Morne Rond - Marigot,A,0.0,0.0,971,0.0
200900072472,2009,1,27,1130,1,1,1,1.0,6.0,25,NaN,A,0.0,0.0,971,0.0
200900072473,2009,1,10,1915,5,1,1,1.0,6.0,15,NaN,A,0.0,0.0,971,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200500087950,2005,12,21,2035,5,2,2,1.0,3.0,416.0,"sans, LEBLOND(RUE M. ET",NaN,NaN,NaN,974,0.0
200500087951,2005,12,23,1010,1,2,1,1.0,3.0,416.0,"SANS, PRESIDENT MITTERAN",NaN,NaN,NaN,974,0.0
200500087952,2005,12,26,1715,1,2,2,1.0,3.0,416.0,"SANS, LEBLOND(RUE M. ET",NaN,NaN,NaN,974,0.0


In [24]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1121571 entries, 200900000001 to 200500087954
Data columns (total 16 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   an            1121571 non-null  int32  
 1   mois          1121571 non-null  int64  
 2   jour          1121571 non-null  int64  
 3   hrmn          1121571 non-null  object 
 4   lum           1121571 non-null  int64  
 5   agg           1121571 non-null  int64  
 6   int           1121571 non-null  int64  
 7   atm           1121498 non-null  float64
 8   col           1121552 non-null  float64
 9   com           1121569 non-null  object 
 10  adr           978295 non-null   object 
 11  gps           480052 non-null   object 
 12  lat           634503 non-null   float64
 13  long          631290 non-null   float64
 14  dep           1121571 non-null  int64  
 15  metropolitan  1121571 non-null  float64
dtypes: float64(5), int32(1), int64(6), object(4)
memory usage: 14

In [25]:
# Look in the data. Look for columns with any NaN first:
features[features.isna().any(axis=1)]

,an,mois,jour,hrmn,lum,agg,int,atm,col,com,adr,gps,lat,long,dep,metropolitan
Num_Acc,,,,,,,,,,,,,,,,
200900000003,2009,1,29,645,3,1,1,1.0,3.0,52,NaN,M,0.0,0.0,44,1.0
200900000007,2009,1,5,1145,1,1,1,2.0,7.0,224,NaN,M,0.0,0.0,44,1.0
200900000008,2009,1,29,1900,3,1,1,1.0,6.0,70,34 RUE DE LA HAUTE CARIZ,M,40.0,NaN,44,1.0
200900000011,2009,1,23,1400,1,2,6,1.0,1.0,154,RD PT ANCIENS COMBATTANT,M,40.0,NaN,44,1.0
200900000015,2009,1,23,745,2,2,1,9.0,6.0,47,121 BD DE LA LIBERATION,M,40.0,NaN,44,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200500087950,2005,12,21,2035,5,2,2,1.0,3.0,416.0,"sans, LEBLOND(RUE M. ET",NaN,NaN,NaN,974,0.0
200500087951,2005,12,23,1010,1,2,1,1.0,3.0,416.0,"SANS, PRESIDENT MITTERAN",NaN,NaN,NaN,974,0.0
200500087952,2005,12,26,1715,1,2,2,1.0,3.0,416.0,"SANS, LEBLOND(RUE M. ET",NaN,NaN,NaN,974,0.0


In [26]:
# Check the GPS variable
features['gps'].unique()

array(['M', nan, 'A', 'R', 'C', 'P', 'S', 'G', 'Y', 'T', '0'],
      dtype=object)

In [27]:
features['dep'].sort_values().unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95, 971, 972, 973, 974, 975, 976, 977, 978, 986,
       987, 988])

The gps variable is not very reliable. We have addresses with "A" located in metropolitan France. Hence, only rely on departments.

### Save to a temporary merged file
This is temporary step to keep the merge file available for the geolocation processing.

In [28]:
features.to_csv("./data/230819_features_merged.csv", sep = ',', header = True, na_rep = 'n/a', index=True)

### Fill gaps in the geolocations
**Credits:** https://github.com/vaclavdekanovsky/data-analysis-in-examples/blob/master/Maps/Geocoding/Address%20to%20Location.ipynb

Get the index of location with existing addresses and no geolocation

Try to retrieve data from OpenStreetMap to fill georeference.

In the case, no data is retrieved, add nan to the georeference.

So far, the retrieval gives up as no valid return occurs. 

Some of the addresses are truncated.

We can try to fill those on a later stage.

Loop the index entries to retrieve datea from [Nominatim](https://nominatim.org).

### Export the data
The data is exported to `features.csv` in the `data` folder.

**Danger!!! May destroy data!**

Load the stored features, if required

In [29]:
features = pd.read_csv("./data/features.csv", header=0, index_col=0, na_values='n/a', low_memory=False)

## `lieux` files
- import the files
- fix `dtype` issues on the fly
- fix issues with latitude and longitude
- export the file to `places.csv` in `data`

### Data import and dtype issues fixing

- encoding is `latin-1` to map Frensh characters


In [30]:
# read places files in data directory
files = []
for file in glob.glob("../mar23cds_accidents_data_import/lieux*.csv"):
    files.append(file)

    # import user files
places = pd.DataFrame()

for i in range(0, len(files)):
    file = pd.read_csv(files[i], sep = ',', encoding='latin-1', na_values=['N/A', 'n/a',''], low_memory=False, index_col=0)
    if i == 0:
        places = file
    else:
        places = pd.concat([places, file], axis = 0)

In [31]:
places.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1121571 entries, 202100000001 to 200700084726
Data columns (total 18 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   catr     1121570 non-null  float64
 1   voie     1014391 non-null  object 
 2   v1       485747 non-null   float64
 3   v2       51369 non-null    object 
 4   circ     1119997 non-null  float64
 5   nbv      1118840 non-null  float64
 6   vosp     1118810 non-null  float64
 7   prof     1119622 non-null  float64
 8   pr       646087 non-null   object 
 9   pr1      644268 non-null   object 
 10  plan     1119290 non-null  float64
 11  lartpc   902739 non-null   object 
 12  larrout  1008730 non-null  object 
 13  surf     1119647 non-null  float64
 14  infra    1116163 non-null  float64
 15  situ     1116601 non-null  float64
 16  vma      163102 non-null   float64
 17  env1     953029 non-null   float64
dtypes: float64(12), object(6)
memory usage: 162.6+ MB


In [32]:
places.head()

,catr,voie,v1,v2,circ,nbv,vosp,prof,pr,pr1,plan,lartpc,larrout,surf,infra,situ,vma,env1
Num_Acc,,,,,,,,,,,,,,,,,,
202100000001,3.0,981,-1.0,NaN,2.0,2.0,0.0,1.0,(1),(1),1.0,NaN,-1,1.0,0.0,1.0,80.0,NaN
202100000002,3.0,20,0.0,E,2.0,2.0,0.0,1.0,0,10,1.0,NaN,-1,1.0,0.0,1.0,80.0,NaN
202100000003,4.0,GENERAL GALLIENI (BOULEVARD DU),0.0,NaN,2.0,2.0,0.0,1.0,(1),(1),1.0,NaN,-1,1.0,0.0,1.0,50.0,NaN
202100000004,3.0,GENERAL GALLIENI (BOULEVARD DU),0.0,NaN,2.0,4.0,0.0,1.0,0,1,1.0,NaN,-1,1.0,0.0,1.0,50.0,NaN
202100000005,7.0,PARIS. ROUTE DE,-1.0,NaN,2.0,2.0,-1.0,1.0,11,150,1.0,NaN,-1,1.0,0.0,1.0,50.0,NaN


### Export the data
The data is exported to `places.csv` in the `data` folder.

In [33]:
places.to_csv("./data/places.csv", sep = ',', header = True, na_rep = 'n/a', index=True)

## `vehicules` files
- import the files
- fix `dtype` issues on the fly
- fix issues with latitude and longitude
- export the file to `vehicles.csv` in `data`

### Data import

In [34]:
# read vehicle files in data directory
files = []
for file in glob.glob("../mar23cds_accidents_data_import/vehicules*.csv"):
    files.append(file)

# import user files
vehicles = pd.DataFrame()

for i in range(0, len(files)):
    file = pd.read_csv(files[i], sep = ',', encoding='utf-8', na_values=['N/A', 'n/a',''], low_memory=False, index_col=0)
    if i == 0:
        vehicles = file
    else:
        vehicles = pd.concat([vehicles, file], axis = 0)

In [35]:
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1914902 entries, 200600000001 to 200900074898
Data columns (total 10 columns):
 #   Column       Dtype  
---  ------       -----  
 0   senc         float64
 1   catv         int64  
 2   occutc       float64
 3   obs          float64
 4   obsm         float64
 5   choc         float64
 6   manv         float64
 7   num_veh      object 
 8   id_vehicule  object 
 9   motor        float64
dtypes: float64(7), int64(1), object(2)
memory usage: 160.7+ MB


In [36]:
vehicles.head()

,senc,catv,occutc,obs,obsm,choc,manv,num_veh,id_vehicule,motor
Num_Acc,,,,,,,,,,
200600000001,0.0,7,0.0,11.0,2.0,6.0,1.0,B01,NaN,NaN
200600000001,0.0,13,0.0,0.0,2.0,1.0,15.0,A01,NaN,NaN
200600000002,0.0,7,0.0,0.0,2.0,3.0,15.0,A01,NaN,NaN
200600000002,0.0,7,0.0,0.0,2.0,3.0,1.0,B01,NaN,NaN
200600000003,0.0,10,0.0,0.0,1.0,4.0,4.0,A01,NaN,NaN


### Fixing the issue in `id_vehicule`

In [37]:
# remove all non-numeric characters by a regex
vehicles['id_vehicule'] = vehicles['id_vehicule'].apply(lambda x: re.sub('[^0-9]', '', str(x)))

# replace empty values by nan
vehicles['id_vehicule'] = vehicles['id_vehicule'].replace('', np.nan).astype(float)

In [38]:
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1914902 entries, 200600000001 to 200900074898
Data columns (total 10 columns):
 #   Column       Dtype  
---  ------       -----  
 0   senc         float64
 1   catv         int64  
 2   occutc       float64
 3   obs          float64
 4   obsm         float64
 5   choc         float64
 6   manv         float64
 7   num_veh      object 
 8   id_vehicule  float64
 9   motor        float64
dtypes: float64(8), int64(1), object(1)
memory usage: 160.7+ MB


### Export the data
The data is exported to `vehicles.csv` in the `data` folder.


In [39]:
vehicles.to_csv("./data/vehicles.csv", sep = ',', header = True, na_rep = 'n/a', index=True)

## `registered vehicules` files
- import the files
- fix `dtype` issues on the fly
- fix issues with latitude and longitude
- export the file to `registered_vehicles.csv` in `data`

### Data import

In [40]:
# read registered vehicle files in data directory
files = []
for file in glob.glob("../mar23cds_accidents_data_import/registered_vehicles*.csv"):
    files.append(file)
# import user files
registered_vehicles = pd.DataFrame()

for i in range(0, len(files)):
    file = pd.read_csv(files[i], sep = ';', na_values=['N/A', 'n/a',''], 
                       low_memory=False, encoding="UTF-8")
    #dtype = {'Age véhicule': 'Int64'})
    if i == 0:
        registered_vehicles = file
    else:
        registered_vehicles = pd.concat([registered_vehicles, file], axis = 0)

registered_vehicles.head()

,Id_accident,Lettre Conventionnelle Véhicule,Année,Lieu Admin Actuel - Territoire Nom,Type Accident - Libellé,CNIT,Catégorie véhicule,Age véhicule
0,63 387 461,A,2019,Métropole,Accident Léger,N10MCDCTPV62189,VU,3.0
1,63 387 461,B,2019,Métropole,Accident Léger,M10JT0VP015D744,VU,2.0
2,63 387 462,A,2019,Métropole,Accident grave non mortel,JS1CJ2251001004,Moto lourde,10.0
3,63 387 463,A,2019,Métropole,Accident Léger,VF7FC8HZB273438,VT,14.0
4,63 387 464,A,2019,Métropole,Accident Léger,M10NSSVP035L180,VT,1.0


In [41]:
registered_vehicles['Age véhicule'] = registered_vehicles['Age véhicule'].replace('#VALEURMULTI', np.nan).astype(float)

In [42]:
registered_vehicles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1290207 entries, 0 to 98024
Data columns (total 8 columns):
 #   Column                              Non-Null Count    Dtype  
---  ------                              --------------    -----  
 0   Id_accident                         1290207 non-null  object 
 1   Lettre Conventionnelle Véhicule     1290206 non-null  object 
 2   Année                               1290207 non-null  int64  
 3   Lieu Admin Actuel - Territoire Nom  1290207 non-null  object 
 4   Type Accident - Libellé             1290207 non-null  object 
 5   CNIT                                845998 non-null   object 
 6   Catégorie véhicule                  1290207 non-null  object 
 7   Age véhicule                        1181107 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 88.6+ MB


### Fixing the issue in `Id_accident`

In [43]:
# remove all non-numeric characters by a regex
registered_vehicles['Id_accident'] = registered_vehicles['Id_accident'].apply(lambda x: re.sub('[^0-9]', '', str(x)))

# replace empty values by nan
registered_vehicles['Id_accident'] = registered_vehicles['Id_accident'].replace('', np.nan).astype(float)

In [44]:
registered_vehicles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1290207 entries, 0 to 98024
Data columns (total 8 columns):
 #   Column                              Non-Null Count    Dtype  
---  ------                              --------------    -----  
 0   Id_accident                         1290207 non-null  float64
 1   Lettre Conventionnelle Véhicule     1290206 non-null  object 
 2   Année                               1290207 non-null  int64  
 3   Lieu Admin Actuel - Territoire Nom  1290207 non-null  object 
 4   Type Accident - Libellé             1290207 non-null  object 
 5   CNIT                                845998 non-null   object 
 6   Catégorie véhicule                  1290207 non-null  object 
 7   Age véhicule                        1181107 non-null  float64
dtypes: float64(2), int64(1), object(5)
memory usage: 88.6+ MB


In [45]:
registered_vehicles.head()

,Id_accident,Lettre Conventionnelle Véhicule,Année,Lieu Admin Actuel - Territoire Nom,Type Accident - Libellé,CNIT,Catégorie véhicule,Age véhicule
0,63387461.0,A,2019,Métropole,Accident Léger,N10MCDCTPV62189,VU,3.0
1,63387461.0,B,2019,Métropole,Accident Léger,M10JT0VP015D744,VU,2.0
2,63387462.0,A,2019,Métropole,Accident grave non mortel,JS1CJ2251001004,Moto lourde,10.0
3,63387463.0,A,2019,Métropole,Accident Léger,VF7FC8HZB273438,VT,14.0
4,63387464.0,A,2019,Métropole,Accident Léger,M10NSSVP035L180,VT,1.0


### Export the data
The data is exported to `registered_vehicles.csv` in the `data` folder.

In [46]:
registered_vehicles.to_csv("./data/registered_vehicles.csv", sep = ',', header = True, na_rep = 'n/a', index=False)

In [47]:
registered_vehicles['Année'].unique()

array([2019, 2020, 2018, 2016, 2010, 2012, 2011, 2013, 2015, 2009, 2021,
       2017, 2014])

In [48]:
registered_vehicles['id_vehicule'] = registered_vehicles['Id_accident']
registered_vehicles = registered_vehicles.drop('Id_accident', axis = 1)

In [49]:
registered_vehicles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1290207 entries, 0 to 98024
Data columns (total 8 columns):
 #   Column                              Non-Null Count    Dtype  
---  ------                              --------------    -----  
 0   Lettre Conventionnelle Véhicule     1290206 non-null  object 
 1   Année                               1290207 non-null  int64  
 2   Lieu Admin Actuel - Territoire Nom  1290207 non-null  object 
 3   Type Accident - Libellé             1290207 non-null  object 
 4   CNIT                                845998 non-null   object 
 5   Catégorie véhicule                  1290207 non-null  object 
 6   Age véhicule                        1181107 non-null  float64
 7   id_vehicule                         1290207 non-null  float64
dtypes: float64(2), int64(1), object(5)
memory usage: 88.6+ MB


### Merge all data
- Merge all files except `registered_vehicles`.
- The variables `id_vehicule` and `num_veh` do exist in `users` and `vehicules`.
- Thus, we need to merge also by these variables to avoid the creation of cross-merged datasets.

In [50]:
# merge files
merged_df = pd.merge(users, features, on='Num_Acc', suffixes=('_users', '_features'))
merged_df = pd.merge(merged_df, places, on='Num_Acc', suffixes=('_merged_df', '_places'))
merged_df = pd.merge(merged_df, vehicles, on=['Num_Acc', 'id_vehicule', 'num_veh'])


In [51]:
merged_df.head()

,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,secu1,secu2,...,vma,env1,senc,catv,occutc,obs,obsm,choc,manv,motor
Num_Acc,,,,,,,,,,,,,,,,,,,,,
201900000001,138306524.0,B01,2.0,2,4,2,2002.0,0.0,1.0,0.0,...,70.0,NaN,2.0,7,NaN,0.0,2.0,5.0,23.0,1.0
201900000001,138306524.0,B01,1.0,1,4,2,1993.0,5.0,1.0,0.0,...,70.0,NaN,2.0,7,NaN,0.0,2.0,5.0,23.0,1.0
201900000001,138306525.0,A01,1.0,1,1,1,1959.0,0.0,1.0,0.0,...,70.0,NaN,2.0,17,NaN,1.0,0.0,3.0,11.0,1.0
201900000002,138306523.0,A01,1.0,1,4,2,1994.0,0.0,1.0,0.0,...,70.0,NaN,1.0,7,NaN,4.0,0.0,1.0,0.0,1.0
201900000003,138306520.0,A01,1.0,1,1,1,1996.0,0.0,1.0,0.0,...,90.0,NaN,1.0,7,NaN,0.0,2.0,1.0,2.0,1.0


In [52]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2509598 entries, 201900000001 to 201800057783
Data columns (total 57 columns):
 #   Column        Dtype  
---  ------        -----  
 0   id_vehicule   float64
 1   num_veh       object 
 2   place         float64
 3   catu          int64  
 4   grav          int64  
 5   sexe          int64  
 6   an_nais       float64
 7   trajet        float64
 8   secu1         float64
 9   secu2         float64
 10  secu3         float64
 11  locp          float64
 12  actp          object 
 13  etatp         float64
 14  secu          float64
 15  an            int64  
 16  mois          int64  
 17  jour          int64  
 18  hrmn          object 
 19  lum           int64  
 20  agg           int64  
 21  int           int64  
 22  atm           float64
 23  col           float64
 24  com           object 
 25  adr           object 
 26  gps           object 
 27  lat           float64
 28  long          float64
 29  dep           int64  
 30  metropo

The registered vehicles does not contain the Index, we need to fix this first before merging

This looks good now.

In [53]:
merged_df.to_csv("./data/231018_merged_tables_basic_data.csv", sep = ',', header = True, na_rep = 'n/a', index=True)